In [1]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Configurations
dataset_folder = "dataset/fruits/apples"
train_folder = f"{dataset_folder}/train"
test_folder = f"{dataset_folder}/test"
img_size = (224, 224)

# Load training and testing datasets
train_dataset = tf.keras.utils.image_dataset_from_directory(train_folder,labels="inferred", label_mode="int", image_size=img_size)
test_dataset = tf.keras.utils.image_dataset_from_directory(test_folder,labels="inferred", label_mode="int", image_size=img_size)

# Configure datasets for performance (buffered prefetching to load images from disk without blocking I/O)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

Found 5334 files belonging to 2 classes.
Found 1334 files belonging to 2 classes.


In [8]:
# Preprocessing method used to rescale pixel values from [0, 255] to [-1, 1]
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
img_shape = img_size + (3,)

# Load pre-trained mobilenet model, don't include top classification layers for better feature extraction (depend on bottleneck layer which retains more generality compared to the top classification layers)
mobilenet = tf.keras.applications.MobileNetV3Small(input_shape=img_shape, include_top=False)

# Prepare training data (extract features)
train_image_batch, train_label_batch = next(iter(train_dataset))
training_features = mobilenet(train_image_batch)

4334752/4334752 [==============================] - 1s 0us/step


In [9]:
# Freeze convolutional base to prevent weights in lower layers from being updated
mobilenet.trainable = False

In [10]:
mobilenet.summary()

Model: "MobilenetV3small"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 224, 224, 3)          0         ['input_2[0][0]']             
                                                                                                  
 Conv (Conv2D)               (None, 112, 112, 16)         432       ['rescaling[0][0]']           
                                                                                                  
 Conv/BatchNorm (BatchNorma  (None, 112, 112, 16)         64        ['Conv[0][0]']                
 lization)                                                                         

In [11]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
training_features_average = global_average_layer(training_features)
print(training_features_average.shape)

(32, 576)


In [ ]:
# Create a custom classification head
model = tf.keras.Sequential([
    mobilenet,
    tf.keras.layers.GlobalAveragePooling2D(),                       # Convert feeatues to a single 1280-element vector per image
    tf.keras.layers.Dense(128, activation='relu'),                  # Add intermediate layer with 1024 input nodes (output from mobilenet model), 128 output nodes, ReLU activation function
    tf.keras.layers.Dense(len(CLASS_NAMES), activation='softmax')   # // Add output layer, with number of neurons equal to number of classification classes, Softmax activation function (for classification problem)
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])